In [39]:
import h5py
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image
import scipy
from scipy.spatial.distance import cdist

In [13]:
hdf5data = h5py.File('../../Data/data_109.h5', 'r')

In [23]:
np.asarray(hdf5data['Sequence'][str(0)]['Detections'])[0]

((-11.14840948, 15.36387945, -0.94192017), 1.17728508, 0.56196702, 1.33484852, 0.6319266)

In [30]:
a = np.array([1,2,3,4,5])
a[np.argmin(np.array([[1,2,3], [3,2, -1], [3,2,-1]]), axis=1)]

array([1, 3, 3])

In [107]:
def link_detections_tracks(detections, tracks):
    
    coord_detections = [np.array(vals[0].tolist()) for vals in detections]
    coord_tracks = [np.array(vals[1].tolist()) for vals in tracks]
    vel_tracks = [vals[2] for vals in tracks]
    track_id = [vals[0] for vals in tracks]
    object_type = [vals[3] for vals in tracks]
    status_tracks = [vals[4] for vals in tracks]
    
    # for the time being, let us associate the closest bounding box, even if it is assigned more than once
    
    dists = cdist(coord_tracks, coord_detections, 'euclidean')
    print(dists)
    associations = np.argmin(dists, axis=1)
    
    associations = associations.tolist()
    associated_detections = detections[associations]
    coord_detections = coord_detections[associations]
    length_box = [vals[1] for vals in associated_detections]
    width_box = [vals[2] for vals in associated_detections]
    height_box = [vals[3] for vals in associated_detections]
    angle_box = [vals[4] for vals in associated_detections]
    
    
    
    returndf = pd.DataFrame()
    returndf["X_track"] = coord_tracks[0]
    returndf["Y_track"] = coord_tracks[1]
    returndf["Z_track"] = coord_tracks[2]
    returndf["vx_track"] = vel_tracks[0]
    returndf["vy_track"] = vel_tracks[1]
    returndf["vz_track"] = vel_tracks[2]
    returndf["track_id"] = track_id
    returndf["obj_type"] = object_type
    returndf["track_status"] = status_tracks
    
    returndf["X_box"] = coord_detections[0]
    returndf["Y_box"] = coord_detections[1]
    returndf["Z_box"] = coord_detections[2]
    returndf["length_box"] = length_box
    returndf["width_box"] = width_box
    returndf["height_box"] = height_box
    returndf["angle_box"] = angle_box
    
    return returndf

In [101]:
def h5_to_df(h5data):
    
    timestamps = h5data['Timestamp']
    df = pd.DataFrame()
    
    for c, t in enumerate(timestamps):
        d = h5data['Sequence'][str(c)]
        track = np.asarray(d['Tracks'])
        detection = np.asarray(d['Detections'])
        row = link_detections_tracks(detection, track)
        append_df = link_detections_tracks(detection, track)
        append_df['timestamp'] = t
        df.append(append_df)
    return df

In [108]:
df = h5_to_df(hdf5data)

[[20.83864637 17.72777091  3.66436921  0.05500284]
 [ 0.03151321  3.96186173 19.35093508 20.79398994]
 [ 3.8168025   0.16081516 16.00058395 17.81576004]
 [ 4.63344793  0.70008351 15.18096399 17.05366337]
 [19.36202414 15.86695341  0.02673758  3.66837846]]
[3, 0, 1, 1, 2]


TypeError: list indices must be integers or slices, not list

In [85]:
c=0
d = hdf5data['Sequence'][str(c)]
track = np.asarray(d['Tracks'])
detection = np.asarray(d['Detections'])

coord_detections = [np.asarray(vals[0].tolist()) for vals in detection]
coord_tracks = [np.asarray(vals[1].tolist()) for vals in track]
dists = cdist(coord_tracks, coord_detections, 'euclidean')
print(type(dists))

<class 'numpy.ndarray'>
